In [1]:
import xarray as xr
import sys 
import pathlib 
import traceback
from rich.console import Console
import dask
import loky
import tqdm
import os

parent_dir = pathlib.Path.cwd().parent
sys.path.append(str(parent_dir))
sys.path.append(str(parent_dir.parent))

from dor_config import DORConfig

from dor_cli import setup_directories
from vis_pyramid import setup_memory, get_nc_glob_pattern
from research_grade_data import open_gx1v7_dataset, process_single_case_no_dask, get_cases_df, get_case_metadata

In [2]:
console = Console()
config = DORConfig()

dirs = setup_directories()
memory = setup_memory(dirs["joblib_cache_dir"])
config

DORConfig(scratch_env='/pscratch/sd/a/abanihi', parent_data_dir='/global/cfs/projectdirs/m4746/Datasets/Ocean-CDR-Atlas-v0/DOR-Efficiency-Map', data_archive_dir='/global/cfs/projectdirs/m4746/Projects/Ocean-CDR-Atlas-v0/data/archive', store_1_path='/pscratch/sd/a/abanihi/test/store1b.zarr', store_2_path='/pscratch/sd/a/abanihi/test/store2.zarr', cumulative_fg_co2_percent_store_path='/pscratch/sd/a/abanihi/test/cumulative_FG_CO2_percent.zarr')

In [3]:
config.compressed_data_dir

PosixPath('/global/cfs/projectdirs/m4746/Datasets/Ocean-CDR-Atlas-v0/DOR-Efficiency-Map/research-grade-compressed/experiments')

In [4]:
df = get_cases_df()
df.head()

,blueprint,polygon,polygon_master,basin,start_date,cdr_forcing,cdr_forcing_file,simulation_key,refdate,stop_n,wallclock,curtail_output
case,,,,,,,,,,,,
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-01-01_00000.001,smyle,0.0,0.0,North_Atlantic_basin,1999-01,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_North_Atlantic_basin_000_1999-01-01_00000,0347-01-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-04-01_00001.001,smyle,0.0,0.0,North_Atlantic_basin,1999-04,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_North_Atlantic_basin_000_1999-04-01_00001,0347-04-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-07-01_00002.001,smyle,0.0,0.0,North_Atlantic_basin,1999-07,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_North_Atlantic_basin_000_1999-07-01_00002,0347-07-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-10-01_00003.001,smyle,0.0,0.0,North_Atlantic_basin,1999-10,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_North_Atlantic_basin_000_1999-10-01_00003,0347-10-01,15,10:00:00,True
smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_001_1999-01-01_00004.001,smyle,1.0,1.0,North_Atlantic_basin,1999-01,DOR,/global/cfs/projectdirs/m4746/Projects/OAE-Eff...,glb-dor_North_Atlantic_basin_001_1999-01-01_00004,0347-01-01,15,10:00:00,True


In [5]:
df.shape

(2760, 12)

In [6]:
case = 'smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-01-01_00000.001'
case_metadata = get_case_metadata(case, df)
case_metadata

blueprint                                                       smyle
polygon                                                           0.0
polygon_master                                                    0.0
basin                                            North_Atlantic_basin
start_date                                                    1999-01
cdr_forcing                                                       DOR
cdr_forcing_file    /global/cfs/projectdirs/m4746/Projects/OAE-Eff...
simulation_key      glb-dor_North_Atlantic_basin_000_1999-01-01_00000
refdate                                                    0347-01-01
stop_n                                                             15
wallclock                                                    10:00:00
curtail_output                                                   True
Name: smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-01-01_00000.001, dtype: object

In [7]:
process_single_case_no_dask(case=case, 
                            case_metadata=case_metadata, 
                            out_path_prefix="/pscratch/sd/a/abanihi/research-grade", 
                            data_dir_path="/global/cfs/projectdirs/m4746/Projects/Ocean-CDR-Atlas-v0/data/archive", verbose=True)

Globbing files (this may take a while)...

Found 180 files for case smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-01-01_00000.001

Processed: /pscratch/sd/a/abanihi/research-grade/000/01/smyle.cdr-atlas-v0.glb-dor.000-1999-01.pop.h.0347-01.nc

Processed: /pscratch/sd/a/abanihi/research-grade/000/01/smyle.cdr-atlas-v0.glb-dor.000-1999-01.pop.h.0347-02.nc

[PosixPath('/pscratch/sd/a/abanihi/research-grade/000/01/smyle.cdr-atlas-v0.glb-dor.000-1999-01.pop.h.0347-01.nc'),
 PosixPath('/pscratch/sd/a/abanihi/research-grade/000/01/smyle.cdr-atlas-v0.glb-dor.000-1999-01.pop.h.0347-02.nc')]

In [8]:
time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)
xr.open_dataset('/pscratch/sd/a/abanihi/research-grade/000/01/smyle.cdr-atlas-v0.glb-dor.000-1999-01.pop.h.0347-01.nc', decode_times=True)

/pscratch/sd/a/abanihi/ipykernel_1410800/769989248.py:2: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  xr.open_dataset('/pscratch/sd/a/abanihi/research-grade/000/01/smyle.cdr-atlas-v0.glb-dor.000-1999-01.pop.h.0347-01.nc', decode_times=True)


<xarray.Dataset> Size: 479MB
Dimensions:                     (intervention_date: 1, elapsed_time: 1,
                                 z_t: 60, z_t_150m: 15, z_w: 60, z_w_top: 60,
                                 z_w_bot: 60, nlat: 384, nlon: 320,
                                 polygon_id: 1)
Coordinates: (12/65)
    time                        (intervention_date, elapsed_time) object 8B ...
  * z_t                         (z_t) float32 240B 500.0 1.5e+03 ... 5.375e+05
  * z_t_150m                    (z_t_150m) float32 60B 500.0 ... 1.45e+04
  * z_w                         (z_w) float32 240B 0.0 1e+03 ... 5e+05 5.25e+05
  * z_w_top                     (z_w_top) float32 240B 0.0 1e+03 ... 5.25e+05
  * z_w_bot                     (z_w_bot) float32 240B 1e+03 2e+03 ... 5.5e+05
    ...                          ...
    nsurface_u                  float64 8B ...
    time_delta                  timedelta64[ns] 8B ...
    area_m2                     (nlat, nlon) float64 983kB ...
  * polygon_id                  (polygon_id) int32 4B 0
  * intervention_date           (intervention_date) object 8B 1999-01-01 00:0...
  * elapsed_time                (elapsed_time) timedelta64[ns] 8B 00:00:00
Dimensions without coordinates: nlat, nlon
Data variables: (12/46)
    CO3                         (polygon_id, intervention_date, elapsed_time, z_t, nlat, nlon) float32 29MB ...
    HCO3                        (polygon_id, intervention_date, elapsed_time, z_t, nlat, nlon) float32 29MB ...
    H2CO3                       (polygon_id, intervention_date, elapsed_time, z_t, nlat, nlon) float32 29MB ...
    pH_3D                       (polygon_id, intervention_date, elapsed_time, z_t, nlat, nlon) float32 29MB ...
    CO3_ALT_CO2                 (polygon_id, intervention_date, elapsed_time, z_t, nlat, nlon) float32 29MB ...
    pH_3D_ALT_CO2               (polygon_id, intervention_date, elapsed_time, z_t, nlat, nlon) float32 29MB ...
    ...                          ...
    STF_ALK_ALT_CO2             (polygon_id, intervention_date, elapsed_time, nlat, nlon) float32 492kB ...
    IAGE                        (polygon_id, intervention_date, elapsed_time, z_t, nlat, nlon) float32 29MB ...
    tend_zint_100m_DIC          (polygon_id, intervention_date, elapsed_time, nlat, nlon) float32 492kB ...
    tend_zint_100m_DIC_ALT_CO2  (polygon_id, intervention_date, elapsed_time, nlat, nlon) float32 492kB ...
    tend_zint_100m_ALK          (polygon_id, intervention_date, elapsed_time, nlat, nlon) float32 492kB ...
    tend_zint_100m_ALK_ALT_CO2  (polygon_id, intervention_date, elapsed_time, nlat, nlon) float32 492kB ...
Attributes:
    title:             smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_19...
    history:           none
    Conventions:       CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-cu...
    time_period_freq:  month_1
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    contents:          Diagnostic and Prognostic Variables
    source:            CCSM POP2, the CCSM Ocean Component
    revision:          $Id$
    calendar:          All years have exactly  365 days.
    start_time:        This dataset was created on 2024-11-04 at 21:08:03.3
    cell_methods:      cell_methods = time: mean ==> the variable values are ...

In [9]:
paths = ["/pscratch/sd/a/abanihi/research-grade/000/01/smyle.cdr-atlas-v0.glb-dor.000-1999-01.pop.h.0347-01.nc", "/pscratch/sd/a/abanihi/research-grade/000/01/smyle.cdr-atlas-v0.glb-dor.000-1999-01.pop.h.0347-02.nc"]
paths

['/pscratch/sd/a/abanihi/research-grade/000/01/smyle.cdr-atlas-v0.glb-dor.000-1999-01.pop.h.0347-01.nc',
 '/pscratch/sd/a/abanihi/research-grade/000/01/smyle.cdr-atlas-v0.glb-dor.000-1999-01.pop.h.0347-02.nc']

In [10]:
case_metadata

blueprint                                                       smyle
polygon                                                           0.0
polygon_master                                                    0.0
basin                                            North_Atlantic_basin
start_date                                                    1999-01
cdr_forcing                                                       DOR
cdr_forcing_file    /global/cfs/projectdirs/m4746/Projects/OAE-Eff...
simulation_key      glb-dor_North_Atlantic_basin_000_1999-01-01_00000
refdate                                                    0347-01-01
stop_n                                                             15
wallclock                                                    10:00:00
curtail_output                                                   True
Name: smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_1999-01-01_00000.001, dtype: object

In [13]:
time_coder = xr.coders.CFDatetimeCoder(use_cftime=True)
ds = xr.open_mfdataset(paths, decode_times=True)
ds

[########################################] | 100% Completed | 103.00 ms
[                                        ] | 0% Completed | 101.66 us

/pscratch/sd/a/abanihi/ipykernel_1410800/4207579166.py:2: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_mfdataset(paths, decode_times=True)
/pscratch/sd/a/abanihi/ipykernel_1410800/4207579166.py:2: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_mfdataset(paths, decode_times=True)


[########################################] | 100% Completed | 101.44 ms
[########################################] | 100% Completed | 100.82 ms
[########################################] | 100% Completed | 101.25 ms
[########################################] | 100% Completed | 100.80 ms
[########################################] | 100% Completed | 100.61 ms
[########################################] | 100% Completed | 100.76 ms
[########################################] | 100% Completed | 100.89 ms
[########################################] | 100% Completed | 101.08 ms
[########################################] | 100% Completed | 100.73 ms
[########################################] | 100% Completed | 100.79 ms
[########################################] | 100% Completed | 101.58 ms
[########################################] | 100% Completed | 100.63 ms
[########################################] | 100% Completed | 100.95 ms
[########################################] | 100% Completed | 10

<xarray.Dataset> Size: 937MB
Dimensions:                     (polygon_id: 1, intervention_date: 1,
                                 elapsed_time: 2, z_t: 60, nlat: 384,
                                 nlon: 320, z_t_150m: 15, z_w: 60, z_w_top: 60,
                                 z_w_bot: 60)
Coordinates: (12/65)
    time                        (intervention_date, elapsed_time) object 16B dask.array<chunksize=(1, 1), meta=np.ndarray>
  * z_t                         (z_t) float32 240B 500.0 1.5e+03 ... 5.375e+05
  * z_t_150m                    (z_t_150m) float32 60B 500.0 ... 1.45e+04
  * z_w                         (z_w) float32 240B 0.0 1e+03 ... 5e+05 5.25e+05
  * z_w_top                     (z_w_top) float32 240B 0.0 1e+03 ... 5.25e+05
  * z_w_bot                     (z_w_bot) float32 240B 1e+03 2e+03 ... 5.5e+05
    ...                          ...
    nsurface_u                  float64 8B 8.297e+04
    time_delta                  (elapsed_time) timedelta64[ns] 16B 31 days 28...
    area_m2                     (nlat, nlon) float64 983kB dask.array<chunksize=(384, 320), meta=np.ndarray>
  * polygon_id                  (polygon_id) int32 4B 0
  * intervention_date           (intervention_date) object 8B 1999-01-01 00:0...
  * elapsed_time                (elapsed_time) timedelta64[ns] 16B 00:00:00 3...
Dimensions without coordinates: nlat, nlon
Data variables: (12/46)
    CO3                         (polygon_id, intervention_date, elapsed_time, z_t, nlat, nlon) float32 59MB dask.array<chunksize=(1, 1, 1, 60, 384, 320), meta=np.ndarray>
    HCO3                        (polygon_id, intervention_date, elapsed_time, z_t, nlat, nlon) float32 59MB dask.array<chunksize=(1, 1, 1, 60, 384, 320), meta=np.ndarray>
    H2CO3                       (polygon_id, intervention_date, elapsed_time, z_t, nlat, nlon) float32 59MB dask.array<chunksize=(1, 1, 1, 60, 384, 320), meta=np.ndarray>
    pH_3D                       (polygon_id, intervention_date, elapsed_time, z_t, nlat, nlon) float32 59MB dask.array<chunksize=(1, 1, 1, 60, 384, 320), meta=np.ndarray>
    CO3_ALT_CO2                 (polygon_id, intervention_date, elapsed_time, z_t, nlat, nlon) float32 59MB dask.array<chunksize=(1, 1, 1, 60, 384, 320), meta=np.ndarray>
    pH_3D_ALT_CO2               (polygon_id, intervention_date, elapsed_time, z_t, nlat, nlon) float32 59MB dask.array<chunksize=(1, 1, 1, 60, 384, 320), meta=np.ndarray>
    ...                          ...
    STF_ALK_ALT_CO2             (polygon_id, intervention_date, elapsed_time, nlat, nlon) float32 983kB dask.array<chunksize=(1, 1, 1, 384, 320), meta=np.ndarray>
    IAGE                        (polygon_id, intervention_date, elapsed_time, z_t, nlat, nlon) float32 59MB dask.array<chunksize=(1, 1, 1, 60, 384, 320), meta=np.ndarray>
    tend_zint_100m_DIC          (polygon_id, intervention_date, elapsed_time, nlat, nlon) float32 983kB dask.array<chunksize=(1, 1, 1, 384, 320), meta=np.ndarray>
    tend_zint_100m_DIC_ALT_CO2  (polygon_id, intervention_date, elapsed_time, nlat, nlon) float32 983kB dask.array<chunksize=(1, 1, 1, 384, 320), meta=np.ndarray>
    tend_zint_100m_ALK          (polygon_id, intervention_date, elapsed_time, nlat, nlon) float32 983kB dask.array<chunksize=(1, 1, 1, 384, 320), meta=np.ndarray>
    tend_zint_100m_ALK_ALT_CO2  (polygon_id, intervention_date, elapsed_time, nlat, nlon) float32 983kB dask.array<chunksize=(1, 1, 1, 384, 320), meta=np.ndarray>
Attributes:
    title:             smyle.cdr-atlas-v0.glb-dor_North_Atlantic_basin_000_19...
    history:           none
    Conventions:       CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-cu...
    time_period_freq:  month_1
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    contents:          Diagnostic and Prognostic Variables
    source:            CCSM POP2, the CCSM Ocean Component
    revision:          $Id$
    calendar:          All years have exactly  365 days.
    start_time:        This dataset was created on 2024-11-04 at 

In [14]:
xr.open_dataset('/global/cfs/projectdirs/m4746/Datasets/Ocean-CDR-Atlas-v0/OAE-Efficiency-Map/research-grade-compressed/experiments/001/01/alk-forcing.001-1999-01.pop.h.0347-01.nc')

/pscratch/sd/a/abanihi/ipykernel_1410800/3133200765.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  xr.open_dataset('/global/cfs/projectdirs/m4746/Datasets/Ocean-CDR-Atlas-v0/OAE-Efficiency-Map/research-grade-compressed/experiments/001/01/alk-forcing.001-1999-01.pop.h.0347-01.nc')


<xarray.Dataset> Size: 327MB
Dimensions:                     (injection_date: 1, elapsed_time: 1, d2: 2,
                                 z_t: 60, z_t_150m: 15, z_w: 60, z_w_top: 60,
                                 z_w_bot: 60, nlat: 384, nlon: 320,
                                 polygon_id: 1)
Coordinates: (12/64)
    time_bound                  (injection_date, elapsed_time, d2) object 16B ...
    time                        (injection_date, elapsed_time) object 8B ...
  * z_t                         (z_t) float32 240B 500.0 1.5e+03 ... 5.375e+05
  * z_t_150m                    (z_t_150m) float32 60B 500.0 ... 1.45e+04
  * z_w                         (z_w) float32 240B 0.0 1e+03 ... 5e+05 5.25e+05
  * z_w_top                     (z_w_top) float32 240B 0.0 1e+03 ... 5.25e+05
    ...                          ...
    sflux_factor                float64 8B ...
    nsurface_t                  float64 8B ...
    nsurface_u                  float64 8B ...
  * polygon_id                  (polygon_id) int64 8B 1
  * injection_date              (injection_date) object 8B 1999-01-01 00:00:00
  * elapsed_time                (elapsed_time) timedelta64[ns] 8B 00:00:00
Dimensions without coordinates: d2, nlat, nlon
Data variables: (12/34)
    CO3                         (polygon_id, injection_date, elapsed_time, z_t, nlat, nlon) float32 29MB ...
    pH_3D                       (polygon_id, injection_date, elapsed_time, z_t, nlat, nlon) float32 29MB ...
    CO3_ALT_CO2                 (polygon_id, injection_date, elapsed_time, z_t, nlat, nlon) float32 29MB ...
    pH_3D_ALT_CO2               (polygon_id, injection_date, elapsed_time, z_t, nlat, nlon) float32 29MB ...
    co3_sat_calc                (polygon_id, injection_date, elapsed_time, z_t, nlat, nlon) float32 29MB ...
    co3_sat_arag                (polygon_id, injection_date, elapsed_time, z_t, nlat, nlon) float32 29MB ...
    ...                          ...
    ALK_ALT_CO2_zint_100m       (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 492kB ...
    STF_ALK_ALT_CO2             (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 492kB ...
    tend_zint_100m_DIC          (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 492kB ...
    tend_zint_100m_DIC_ALT_CO2  (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 492kB ...
    tend_zint_100m_ALK          (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 492kB ...
    tend_zint_100m_ALK_ALT_CO2  (polygon_id, injection_date, elapsed_time, nlat, nlon) float32 492kB ...
Attributes:
    title:             smyle-fosi.North_Atlantic_basin.alk-forcing-North_Atla...
    history:           none
    Conventions:       CF-1.0; http://www.cgd.ucar.edu/cms/eaton/netcdf/CF-cu...
    time_period_freq:  month_1
    model_doi_url:     https://doi.org/10.5065/D67H1H0V
    contents:          Diagnostic and Prognostic Variables
    source:            CCSM POP2, the CCSM Ocean Component
    revision:          $Id$
    calendar:          All years have exactly  365 days.
    start_time:        This dataset was created on 2023-05-29 at 09:23:25.0
    cell_methods:      cell_methods = time: mean ==> the variable values are ...